In [1]:
#Instalar dependencias
!pip install mlflow pyngrok --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.2/700.2 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [3]:
#Iniciar MLflow con ngrok
import os
from pyngrok import ngrok


# Autenticarse con ngrok
ngrok.set_auth_token("2wzPMwd9GV5Gmt8tAb8x3beLz2o_7Vr99HUFWwZjBJudxs13w")

# Crear túnel al puerto 5000 donde corre MLflow
public_url = ngrok.connect(5000)
print(f"MLflow UI disponible en: {public_url}")

# 7. Lanzar la interfaz web de MLflow en segundo plano
os.system("mlflow ui --port 5000 &")

MLflow UI disponible en: NgrokTunnel: "https://fdb4-34-16-226-200.ngrok-free.app" -> "http://localhost:5000"


0

In [4]:
#Preparar datos y modelo
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

# Cargar y procesar datos
emails = pd.read_csv('/content/emails.csv', index_col=0)
emails['Prediction'] = emails['Prediction'].map({'no_spam': 0, 'spam': 1})
df = emails.copy()
df_train = df.sample(frac=0.8, random_state=0)
df_valid = df.drop(df_train.index)

X_train = df_train.drop('Prediction', axis=1)
y_train = df_train['Prediction']
X_valid = df_valid.drop('Prediction', axis=1)
y_valid = df_valid['Prediction']

# Codificar etiquetas por si acaso
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_valid = le.transform(y_valid)

input_dim = X_train.shape[1]

In [5]:
#Entrenar modelo y registrar en MLflow
import mlflow
import mlflow.tensorflow

mlflow.set_experiment("Email Spam Detection")

with mlflow.start_run():
    model = keras.Sequential([
        layers.BatchNormalization(input_shape=[input_dim]),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

    early_stopping = keras.callbacks.EarlyStopping(
        patience=10, min_delta=0.001, restore_best_weights=True
    )

    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        batch_size=512,
        epochs=1000,
        callbacks=[early_stopping],
        verbose=0
    )

    # Guardar métricas
    val_loss = np.min(history.history['val_loss'])
    val_accuracy = np.max(history.history['val_binary_accuracy'])

    mlflow.log_metric("best_val_loss", val_loss)
    mlflow.log_metric("best_val_accuracy", val_accuracy)

    # Guardar modelo
    mlflow.tensorflow.log_model(model, artifact_path="model")

    print(f"Modelo registrado en MLflow\nVal Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")


2025/05/17 10:08:31 INFO mlflow.tracking.fluent: Experiment with name 'Email Spam Detection' does not exist. Creating a new experiment.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/normalization/batch_normalization.py:142: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025/05/17 10:10:55 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/05/17 10:11:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo registrado en MLflow
Val Loss: 0.0022, Val Accuracy: 1.0000


In [6]:
# Autenticarse con ngrok
ngrok.set_auth_token("2wzPMwd9GV5Gmt8tAb8x3beLz2o_7Vr99HUFWwZjBJudxs13w")

# Crear túnel al puerto 5000 donde corre MLflow
public_url = ngrok.connect(5000)
print(f"MLflow UI disponible en: {public_url}")

# 7. Lanzar la interfaz web de MLflow en segundo plano
os.system("mlflow ui --port 5000 &")

MLflow UI disponible en: NgrokTunnel: "https://0c46-34-16-226-200.ngrok-free.app" -> "http://localhost:5000"


0